At first we load the data and do the pre-processing on it 
1. check if there is any NULL in the dataset (Because it is the MNIST dataset and it is popular we did not check this part)
2. normalize the dataset with z-score and transfer the dataset to the new space with the mean of zero and the standard deviation of 1
(Normalizing is not necessary in this project because all the features are in the same range from 0 to 255 but we do not do it here because it cause a problem in the fiting of the autoencoder and it needs integers)

In [1]:
import pandas as pd
from scipy.stats import zscore

# Load the Data Set
data_labeled_train = pd.read_csv('labeled_train_set.csv')
data_labeled_test = pd.read_csv('test_set.csv')
data_unlabeled_train = pd.read_csv('unlabeled_train_set.csv')

# Conver the Data Set to Numpy Array and Spliting Labels and Features
X_train_labeled = data_labeled_train.iloc[: , 1:].values
y_train_labeled = data_labeled_train.iloc[: , 0].values
X_test_labeled = data_labeled_test.iloc[: , 1:].values
y_test_labeled = data_labeled_test.iloc[: , 0].values
X_train_unlabeled = data_unlabeled_train.iloc[:, :].values


Now we build an Auto Encoder and use it for Feature Selection and by that Reduce the dimentionality of the data
We are going to implement it in 5 layers: 
1. Input Layer
2. 3 Hidden Layers (128 , 16, 128)
3. Output Layer
So now it is obvious how the architecture of our Encoder and also Decoder is

In [30]:
from tensorflow import keras
from tensorflow.keras import layers

# Implimenting the Auto Encoder with the architecture we talked about

# Defining the Input Image dimention
input_image = keras.Input(shape=(784,))

# Define the Encoder
encoded = layers.Dense(128, activation='relu')(input_image)
encoded = layers.Dense(16, activation='relu')(encoded)

# Build the Encoder
encoder = keras.Model(input_image, encoded)

# Define the Decoder 
encoded_input = keras.Input(shape=(16,))
decoded = layers.Dense(128, activation='relu')(encoded_input)
decoded = layers.Dense(784, activation='sigmoid')(decoded)

# Build the Decoder 
decoder = keras.Model(encoded_input, decoded)

# Build the Auto Encoder
auto_encoder = keras.Model(input_image, decoder(encoder(input_image)))

# Compile the Auto Encoder
auto_encoder.compile(optimizer="sgd", loss="mean_squared_error")

# Fit the Data on our Auto Encoder
log = auto_encoder.fit(X_train_labeled, X_train_labeled, epochs=50, batch_size=256,
shuffle=True, validation_data=(X_test_labeled, X_test_labeled))
    

Epoch 1/50
71/71 [==============================] - 2s 20ms/step - loss: 7231.4326 - val_loss: 7348.8545
Epoch 2/50
71/71 [==============================] - 1s 11ms/step - loss: 7230.2241 - val_loss: 7348.8232
Epoch 3/50
71/71 [==============================] - 1s 10ms/step - loss: 7230.2070 - val_loss: 7348.8145
Epoch 4/50
71/71 [==============================] - 1s 10ms/step - loss: 7230.2026 - val_loss: 7348.8096
Epoch 5/50
71/71 [==============================] - 1s 15ms/step - loss: 7230.1992 - val_loss: 7348.8066
Epoch 6/50
71/71 [==============================] - 1s 15ms/step - loss: 7230.1948 - val_loss: 7348.8022
Epoch 7/50
71/71 [==============================] - 1s 15ms/step - loss: 7230.1899 - val_loss: 7348.7974
Epoch 8/50
71/71 [==============================] - 1s 13ms/step - loss: 7230.1880 - val_loss: 7348.7959
Epoch 9/50
71/71 [==============================] - 1s 9ms/step - loss: 7230.1855 - val_loss: 7348.7944
Epoch 10/50
71/71 [==============================] - 1s 

After Fitting of the Auto Encoder is done, we encode the data set with it (reduce its dimentionality to 16)
P.S. I decide to reduce the dimentionality of the unlabeled data and then implement clustring algorithm on it to decrease the time of the running time

In [31]:
encoded_X_train_labeled = encoder.predict(X_train_labeled)
encoded_X_test_labeled = encoder.predict(X_test_labeled)
encoded_X_train_unlabeled = encoder.predict(X_train_unlabeled)


1313/1313 [==============================] - 4s 3ms/step


Before building the MLP model, I want to cluster the unlabeled data
Clustring Algorithm : K-Means

In [32]:
from sklearn.cluster import KMeans
import numpy as np

# Build the Model
k_means = KMeans(n_clusters=10)

# Fit Model on the Unlabeled Data
k_means.fit(encoded_X_train_unlabeled)

# Get the Labels
labels = np.array(k_means.labels_)


e:\tuts\DataMining\Homeworks\HW3\AutoEncoder\env\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Post-processing 
After the clustering we have to know that which label correspond to which number from 0 to 9,
So I decide to get a sample from all of the clusters of size 10 and visually decide which label belongs to which number

In [60]:
# Get sample from all of the labels of size 10
instances_features = []
for index in range(10):
    count = 0
    for i in range(len(labels)):
        if labels[i] == index and count < 10:
            instances_features.append(X_train_unlabeled[i])
            count += 1


# Reshaping the Matrix to prepare it for the image building
print(instances_features[0])
print(len(instances_features))

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0  39 125 225 254
 254 255 254 238 125  51   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0 206 253 253 253 253 253 253 253 253 173  51   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  38 228 253
 253 253 253 253 253 253 253 253 240  51   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0  85 253 253 172 119  19  19  39 162 253 253
 253 237   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   9
 132  52   6   0   0   0   0   4 166 253 253 247   

With Pillow Library, I convert the matrixes to images

In [61]:
from PIL import Image

for i in range(len(instances_features)):
    instances_features[i] = np.reshape(instances_features[i],(28, 28))
    instances_features[i] = instances_features[i].astype(np.uint8)
    img = Image.fromarray(instances_features[i], mode='L')
    img.save(f'img{i}.png')
    img.show()

Now we Have our images and can visually recognize that which label refers to which number 
label 0 --> number 8
label 1 --> number 4
label 2 --> number 3
label 3 --> number 7
label 4 --> number 0
label 5 --> number 2
label 6 --> number 6
label 7 --> number 9
label 8 --> number 1
label 9 --> number 5

P.S. This little Sample did not give me the proper result I had to visuallize more samples from each label but for the report i just show the first 100 instances that was my first sample

Post-processing of the clustring is in the below section 

In [ ]:
# Changing the labels to the numbers that belongs to that cluster
for i in range(len(labels)):
    if labels[i] == 0:
        labels[i] = 8
    elif labels[i] == 1:
        labels[i] = 4
    elif labels[i] == 2:
        labels[i] = 3
    elif labels[i] == 3:
        labels[i] = 7
    elif labels[i] == 4:
        labels[i] = 0
    elif labels[i] == 5:
        labels[i] = 2
    elif labels[i] == 6:
        labels[i] = 6
    elif labels[i] == 7:
        labels[i] = 9
    elif labels[i] == 8:
        labels[i] = 1
    elif labels[i] == 9:
        labels[i] = 5